In [1]:
library(dplyr)
library(xgboost)
library(reshape2)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice



In [2]:
data = read.csv("0914_categorized.csv")

In [4]:
data = data[,-1]

In [8]:
data1 = data  %>% filter(length == 1) %>% unique()

In [9]:
wide1 = data1  %>% group_by(acc_id) %>% mutate(index = row_number()) %>% as.data.frame()
wide1 = reshape(wide1, idvar = "acc_id", timevar = "index", direction = "wide")

In [10]:
id_1  = wide1 %>% dplyr::select(acc_id)

In [11]:
wide1 = wide1 %>% dplyr::select(-wk.1, -acc_id)

In [12]:
wide1$label.1 = as.integer(wide1$label.1) - 1
wide1$label.1[which(wide1$label.1 == 4)] = 3
ans1          = wide1$label.1
wide1         = wide1 %>% dplyr::select(-label.1)
data_matrix1  = xgb.DMatrix(data  = as.matrix(wide1), 
                              label = ans1)

In [13]:
# XGboost (param 조정해야하는 거 많음)
## User Function ##
f1_calcul = function(table_eval){
  {
    fffmea = 8 / {
      1 / (table_eval[1,1] / sum(table_eval[,1])) +
        1 / (table_eval[2,2] / sum(table_eval[,2])) +
        1 / (table_eval[3,3] / sum(table_eval[,3])) +
        1 / (table_eval[4,4] / sum(table_eval[,4])) +
        1 / (table_eval[1,1] / sum(table_eval[1,])) +
        1 / (table_eval[2,2] / sum(table_eval[2,])) +
        1 / (table_eval[3,3] / sum(table_eval[3,])) +
        1 / (table_eval[4,4] / sum(table_eval[4,]))
    } 
    return(fffmea)
  }}

In [14]:
as.numeric.factor = function(x) {as.numeric(levels(x))[x]}

f1.max = function(preds, dtrain) {
  labels = getinfo(dtrain, "label")
  preds = matrix(preds, ncol=4, byrow=TRUE)
  table_eval = table(apply(preds,1,which.max)-1, labels)
  err = f1_calcul(table_eval)
  return(list(metric = "f1", value = err))
}

In [15]:
xgb_params1 = list(objective = "multi:softprob",
                     eval_metric = f1.max,
                     num_class   = 4,
                     max_depth   = 25,                       
                     eta         = 0.08,                     
                     gamma       = 0.3,
                     lamda       = 0.0,
                     alpha       = 1.0,
                     max_delta_step = 5,                    
                     colsample_bytree = 0.8,
                     subsample = 0.8,
                     min_child_weight = 5 )

In [ ]:
a1    = xgb.cv(params = xgb_params1,
                data = data_matrix1,
                label = ans1,
                nrounds = 1000,
                prediction = TRUE,
                nfold = 5,
                maximize = TRUE,
                print_every_n = 10,
                early_stopping_rounds = 20,
                verbose = 1,
                callback(cb.cv.predict(save_models = TRUE)))

Warning message in xgb.get.DMatrix(data, label, missing):
“xgboost: label will be ignored.”

In [17]:
a1$best_iteration

[1] 109